In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [5]:
cwl_dir = '/home/estorrs/pecgs-pipeline/cwl/align_dnaseq'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

looking at another template for reference

In [8]:
t = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/gdc-dnaseq-cwl/example/gdc_dnaseq_main_workflow_example_input.yaml'))
t

{'bam_name': 'desired-output-name.bam',
 'job_uuid': 'example-test-uuid',
 'collect_wgs_metrics': True,
 'amplicon_kit_set_file_list': [],
 'capture_kit_set_file_list': [],
 'readgroup_fastq_pe_file_list': [{'forward_fastq': {'class': 'File',
    'location': 'input_R1.fastq.gz'},
   'reverse_fastq': {'class': 'File', 'location': 'input_R2.fastq.gz'},
   'readgroup_meta': {'ID': 'input_id',
    'PL': 'ILLUMINA',
    'SM': 'input_sample_id'}}],
 'readgroup_fastq_se_file_list': [],
 'readgroups_bam_file_list': [],
 'common_biallelic_vcf': {'class': 'File',
  'location': 'somatic-hg38_small_exac_common_3.hg38.vcf.gz',
  'secondaryFiles': [{'class': 'File',
    'location': 'somatic-hg38_small_exac_common_3.hg38.vcf.gz.tbi'}]},
 'known_snp': {'class': 'File',
  'location': 'dbsnp_144.hg38.vcf.gz',
  'secondaryFiles': [{'class': 'File',
    'location': 'dbsnp_144.hg38.vcf.gz.tbi'}]},
 'run_markduplicates': True,
 'reference_sequence': {'class': 'File',
  'location': 'GRCh38.d1.vd1.fa',
  'sec

test invocation

In [11]:
template = {
    'bam_name': 'a_string',
    'job_uuid': 'a_string',
    'thread_count': 'a_int',
    'known_snp': {
        'class': 'File',
        'path': 'path/to/vcf.gz',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/vcf.gz.tbi'}
        ]
    },
    'reference_sequence': {
        'class': 'File',
        'path': 'path/to/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.dict'}
        ]
    },
    'readgroup_fastq_pe_file_list': [
        {
            'forward_fastq': {
                'class': 'File',
                'path': 'path/to/R1/fastq'
            },
            'reverse_fastq': {
                'class': 'File',
                'path': 'path/to/R2/fastq'
            },
            'readgroup_meta': {
                'ID': 'input_id',
                'PL': 'ILLUMINA',
                'SM': 'input_sample_id'
            }
        }
    ]
}

In [12]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.align_dnaseq_workflow.yaml'), 'w'))

make cwl file

In [3]:
t = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/gdc-dnaseq-cwl/subworkflows/main/gdc_dnaseq_main_workflow.cwl'))
t

{'cwlVersion': 'v1.0',
 'class': 'Workflow',
 'id': 'gdc_dnaseq_main_wf',
 'requirements': [{'class': 'InlineJavascriptRequirement'},
  {'class': 'MultipleInputFeatureRequirement'},
  {'class': 'ScatterFeatureRequirement'},
  {'class': 'SchemaDefRequirement',
   'types': [{'$import': '../../tools/target_kit_schema.yml'},
    {'$import': '../../tools/readgroup.yml'}]},
  {'class': 'StepInputExpressionRequirement'},
  {'class': 'SubworkflowFeatureRequirement'}],
 'inputs': {'bam_name': 'string',
  'job_uuid': 'string',
  'collect_wgs_metrics': 'boolean',
  'amplicon_kit_set_file_list': {'type': {'type': 'array',
    'items': '../../tools/target_kit_schema.yml#amplicon_kit_set_file'}},
  'capture_kit_set_file_list': {'type': {'type': 'array',
    'items': '../../tools/target_kit_schema.yml#capture_kit_set_file'}},
  'readgroup_fastq_pe_file_list': {'type': {'type': 'array',
    'items': '../../tools/readgroup.yml#readgroup_fastq_file'}},
  'readgroup_fastq_se_file_list': {'type': {'type':

In [6]:
cwl = {
    'cwlVersion': 'v1.0',
    'class': 'Workflow',
    'id': 'align_dnaseq',
    'label': 'align_dnaseq',
    'inputs': {
        'bam_name': 'string',
        'job_uuid': 'string',
        'readgroup_fastq_pe_file_list': {
            'type': {
                'type': 'array',
                'items': '../../submodules/gdc-dnaseq-cwl/tools/readgroup.yml#readgroup_fastq_file'
            }
        },
       'known_snp': {
           'type': 'File',
           'secondaryFiles': ['.tbi']
       },
       'run_markduplicates': {
           'type': 'boolean?',
           'default': True
       },
       'reference_sequence': {
           'type': 'File',
           'secondaryFiles': [
               '.amb',
               '.ann',
               '.bwt',
               '.fai',
               '.pac',
               '.sa',
               '^.dict'
           ]
       },
       'thread_count': 'long'
    },
    'outputs': {
        'output_bam': {
            'type': 'File',
            'outputSource': 'gatk_applybqsr/output_bam'
        }
    },
    'steps': {
        'fastq_clean_pe': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/utils/fastq_clean.cwl',
            'scatter': 'input',
            'in': {'input': 'readgroup_fastq_pe_file_list', 'job_uuid': 'job_uuid'},
            'out': ['output']
        },
        'fastq_clean_se': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/utils/fastq_clean.cwl',
            'scatter': 'input',
            'in': {'input': 'readgroup_fastq_se_file_list', 'job_uuid': 'job_uuid'},
            'out': ['output']
        },
        'readgroups_bam_to_readgroups_fastq_lists': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/utils/readgroups_bam_to_readgroups_fastq_lists.cwl',
            'scatter': 'readgroups_bam_file',
            'in': {'readgroups_bam_file': 'readgroups_bam_file_list'},
            'out': ['pe_file_list', 'se_file_list', 'o1_file_list', 'o2_file_list']
        },
        'merge_fastq_arrays': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/utils/merge_fastq_array_workflow.cwl',
            'in': {
                'bam_pe_fastqs': 'readgroups_bam_to_readgroups_fastq_lists/pe_file_list',
                'bam_se_fastqs': 'readgroups_bam_to_readgroups_fastq_lists/se_file_list',
                'bam_o1_fastqs': 'readgroups_bam_to_readgroups_fastq_lists/o1_file_list',
                'bam_o2_fastqs': 'readgroups_bam_to_readgroups_fastq_lists/o2_file_list',
                'fastqs_pe': 'fastq_clean_pe/output',
                'fastqs_se': 'fastq_clean_se/output'
            },
            'out': ['merged_pe_fastq_array', 'merged_se_fastq_array']
        },
        'bwa_pe': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/align/bwa_pe.cwl',
            'scatter': 'readgroup_fastq_pe',
            'in': {
                'job_uuid': 'job_uuid',
                'reference_sequence': 'reference_sequence',
                'readgroup_fastq_pe': 'merge_fastq_arrays/merged_pe_fastq_array',
                'thread_count': 'thread_count'
            },
            'out': ['bam']
        },
        'bwa_se': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/align/bwa_se.cwl',
            'scatter': 'readgroup_fastq_se',
            'in': {
                'job_uuid': 'job_uuid',
                'reference_sequence': 'reference_sequence',
                'readgroup_fastq_se': 'merge_fastq_arrays/merged_se_fastq_array',
                'thread_count': 'thread_count'
            },
            'out': ['bam']
        },
        'dup_branch_decider': {
            'run': '../../submodules/gdc-dnaseq-cwl/tools/decider_markdup_input.cwl',
            'in': {
                'run_markdups': 'run_markduplicates',
                'bam': {'source': ['bwa_pe/bam', 'bwa_se/bam']}
            },
            'out': ['do_markdup_workflow', 'skip_markdup_workflow', 'out_bam']
        },
        'conditional_markduplicates': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/utils/conditional_markduplicates.cwl',
            'scatter': 'run_markduplicates',
            'in': {
                'bam': 'dup_branch_decider/out_bam',
                'job_uuid': 'job_uuid',
                'thread_count': 'thread_count',
                'bam_name': 'bam_name',
                'run_markduplicates': 'dup_branch_decider/do_markdup_workflow'
            },
            'out': ['output', 'sqlite']
        },
        'conditional_skip_markduplicates': {
            'run': '../../submodules/gdc-dnaseq-cwl/subworkflows/utils/conditional_skip_markduplicates.cwl',
            'scatter': 'skip_markduplicates',
            'in': {
                'bam': 'dup_branch_decider/out_bam',
                'job_uuid': 'job_uuid',
                'thread_count': 'thread_count',
                'bam_name': 'bam_name',
                'skip_markduplicates': 'dup_branch_decider/skip_markdup_workflow'
            },
            'out': ['output']
        },
        'dup_outputs_decider': {
            'run': '../../submodules/gdc-dnaseq-cwl/tools/decider_markdup_output.cwl',
            'in': {
                'markdup_bam': 'conditional_markduplicates/output',
                'markdup_sqlite': 'conditional_markduplicates/sqlite',
                'skip_markdup_bam': 'conditional_skip_markduplicates/output'
            },
            'out': ['bam']
        },
        'gatk_baserecalibrator': {
            'run': '../../submodules/gdc-dnaseq-cwl/tools/gatk4_baserecalibrator.cwl',
            'in': {
                'input': 'dup_outputs_decider/bam',
                'known-sites': 'known_snp',
                'reference': 'reference_sequence'
            },
            'out': ['output_grp']
        },
        'gatk_applybqsr': {
            'run': '../../submodules/gdc-dnaseq-cwl/tools/gatk4_applybqsr.cwl',
            'in': {'input': 'dup_outputs_decider/bam', 'bqsr-recal-file': 'gatk_baserecalibrator/output_grp'},
            'out': ['output_bam']
        },
        'picard_validatesamfile_bqsr': {
            'run': '../../submodules/gdc-dnaseq-cwl/tools/picard_validatesamfile.cwl',
            'in': {'INPUT': 'gatk_applybqsr/output_bam', 'VALIDATION_STRINGENCY': {'valueFrom': 'STRICT'}},
            'out': ['OUTPUT']
        },
    },
   'requirements': [
       {'class': 'InlineJavascriptRequirement'},
       {'class': 'MultipleInputFeatureRequirement'},
       {'class': 'ScatterFeatureRequirement'},
       {
           'class': 'SchemaDefRequirement',
           'types': [
               {'$import': '../../submodules/gdc-dnaseq-cwl/tools/readgroup.yml'}
           ]
       },
      {'class': 'StepInputExpressionRequirement'},
      {'class': 'SubworkflowFeatureRequirement'}
   ]
}

In [7]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'align_dnaseq_workflow.cwl'), 'w'))